# Customer-product dataset

Here we show how querying a customer-product type of dataset may be easier with a graph data model vs tabular data model. [Inspiration](https://neo4j.com/developer/cypher/guide-sql-to-cypher/#cypher-sql-northwind)

### Tabular data model

![](assets/product-category-supplier.png)

### Graph data model

![](assets/product-graph.png)

## 0. Preliminaries

- SQL: use Northwind loaded in at [DB Fiddle](https://www.db-fiddle.com/f/ki1ztAsMywXbkU8hjEH4Rc/0) with PostgreSQL ([source](https://docs.yugabyte.com/preview/sample-data/northwind/))
- Neo4j: load Northwind using the [guide](https://guides.neo4j.com/northwind/index.html); data are provided in `northwind/data`

## 1. What category is each product in?

**SQL**

```sql
SELECT p.product_name, c.category_name 
FROM products as p
JOIN categories as c
ON c.category_id = p.category_id
ORDER BY p.product_name LIMIT 10;
```

**Cypher**

```cypher
MATCH (p:Product) -[:PART_OF]-> (c:Category)
RETURN p.productName, c.categoryName ORDER BY p.productName LIMIT 10
```

Result:

| product_name                 | category_name  |
| ---------------------------- | -------------- |
| Alice Mutton                 | Meat/Poultry   |
| Aniseed Syrup                | Condiments     |
| Boston Crab Meat             | Seafood        |
| Camembert Pierrot            | Dairy Products |
| Carnarvon Tigers             | Seafood        |
| Chai                         | Beverages      |
| Chang                        | Beverages      |
| Chartreuse verte             | Beverages      |
| Chef Anton's Cajun Seasoning | Condiments     |
| Chef Anton's Gumbo Mix       | Condiments     |


## 2. Which categories does each supplier supply?

**SQL**

```sql
SELECT s.company_name, string_agg(DISTINCT c.category_name, ', ') AS Categories
FROM products as p
JOIN categories as c ON c.category_id = p.category_id
JOIN suppliers as s ON s.supplier_id = p.supplier_id
GROUP BY s.company_name 
ORDER BY s.company_name LIMIT 10;
```

**Cypher**
```cypher
MATCH (s:Supplier)-->(p:Product)-->(c:Category)
RETURN s.companyName, collect(DISTINCT c.categoryName) as Categories 
ORDER BY s.companyName LIMIT 10
```

Result:

| company_name                       | categories                            |
| ---------------------------------- | ------------------------------------- |
| Aux joyeux ecclésiastiques         | Beverages                             |
| Bigfoot Breweries                  | Beverages                             |
| Cooperativa de Quesos 'Las Cabras' | Dairy Products                        |
| Escargots Nouveaux                 | Seafood                               |
| Exotic Liquids                     | Beverages, Condiments                 |
| Formaggi Fortini s.r.l.            | Dairy Products                        |
| Forêts d'érables                   | Condiments, Confections               |
| G'day, Mate                        | Grains/Cereals, Meat/Poultry, Produce |
| Gai pâturage                       | Dairy Products                        |
| Grandma Kelly's Homestead          | Condiments, Produce                   |

## 3. Further visualisation

Introducing `Order` table

```cypher
MATCH (p:Product)<-[:ORDERS]-(o:Order)<-[:PURCHASED]-(c:Customer)
RETURN p, o, c LIMIT 50
```

![](assets/sample.png)

```cypher
MATCH (p:Product)<-[:ORDERS]-(o:Order)<-[:PURCHASED]-(c:Customer)
RETURN p, o, c LIMIT 200
```

![](assets/sample_200.png)